# Exercise 1
Author: Tobias Famos

# Importing the needed Packages

In [1]:
import pandas
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from mlxtend.evaluate import accuracy_score
from mlxtend.classifier import OneRClassifier

# Task 1
First lets define a function to evaluate prediction rules. Prediction rules will be give and a lambda function doing it's decision based only on a single row.

￼
Markdown
## Loading the Data

In [2]:
df = pandas.read_csv("Data/titanic.csv")
df.head(10)

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
5,0,3,Mr. James Moran,male,27.0,0,0,8.4583
6,0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
7,0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.0750
8,1,3,Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson,female,27.0,0,2,11.1333
9,1,2,Mrs. Nicholas (Adele Achem) Nasser,female,14.0,1,0,30.0708


In [3]:
def is_correct_prediction(row, prediction_name):
    if row['Survived'] == row[prediction_name]:
        return True
    else:
        return False

def predict_and_evaluate(func):
    df['prediction'] = df.apply (lambda row: func(row), axis=1)
    correct_predictions = df.apply(lambda row: is_correct_prediction(row, 'prediction'), axis=1)
    accuracy =  correct_predictions.value_counts()[True] / correct_predictions.size
    print(f"Accuracy for {func.__name__}: {accuracy}")
    return accuracy


## Task 1.a) What is the best default rule for this dataset?
There are two possible rules without any prior knowledge of the person:
- survived = yes
- survived = no;
Let's count the values and see which brings the higher accuracy.

def label_all_died(row):
    return 0

def label_all_survived(row):
    return 1

predict_and_evaluate(label_all_died)
predict_and_evaluate(label_all_survived)



As one can see from the output above, the best accuracy is when we use the default rule survived = 0.

## Task 1.b)
What is the best 1R for this dataset?
Lets first import our OneRClassifier

Define a function to evaluate a OneR based on one attribute

In [4]:
def create_and_evaluate_1R_for_attribute(attribute):
    x_d = df[[attribute]]
    y = df["Survived"]
    x_d_train, x_d_test, y_train, y_test = train_test_split(x_d, y, test_size=0.2)
    one_r = OneRClassifier()
    one_r.fit(x_d_train.to_numpy(), y_train)
    y_pred = one_r.predict(x_d_test.to_numpy())
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for 1R based on attribute {attribute}: {accuracy}')

Evaluate all the possible Attributes

In [5]:
create_and_evaluate_1R_for_attribute('Pclass')
create_and_evaluate_1R_for_attribute('Sex')
create_and_evaluate_1R_for_attribute('Age')
create_and_evaluate_1R_for_attribute('Siblings/Spouses Aboard')
create_and_evaluate_1R_for_attribute('Parents/Children Aboard')
create_and_evaluate_1R_for_attribute('Fare')


Accuracy for 1R based on attribute Pclass: 0.7191011235955056
Accuracy for 1R based on attribute Sex: 0.797752808988764
Accuracy for 1R based on attribute Age: 0.6348314606741573
Accuracy for 1R based on attribute Siblings/Spouses Aboard: 0.651685393258427
Accuracy for 1R based on attribute Parents/Children Aboard: 0.6460674157303371
Accuracy for 1R based on attribute Fare: 0.6910112359550562


So the best 1R Rule for this datasets seems to be based on sex. Let's write the predictor just for fun and see the error throughout the whole dataset.

In [6]:
create_and_evaluate_1R_for_attribute('Fare')


Accuracy for 1R based on attribute Fare: 0.7134831460674157


In [7]:
def one_rule_sex(row):
    if row['Sex'] == 'female':
        return 1
    return 0
predict_and_evaluate(one_rule_sex)

Accuracy for one_rule_sex: 0.7857948139797069


0.7857948139797069

## Task 1.c) Can you produce a second rule based on a single attribute with a good effectiveness?

Sure, we will just take the second highest ranking rule above.

In [8]:
create_and_evaluate_1R_for_attribute('Pclass')
create_and_evaluate_1R_for_attribute('Sex')
create_and_evaluate_1R_for_attribute('Age')
create_and_evaluate_1R_for_attribute('Siblings/Spouses Aboard')
create_and_evaluate_1R_for_attribute('Parents/Children Aboard')
create_and_evaluate_1R_for_attribute('Fare')

Accuracy for 1R based on attribute Pclass: 0.7078651685393258
Accuracy for 1R based on attribute Sex: 0.8089887640449438
Accuracy for 1R based on attribute Age: 0.5617977528089888
Accuracy for 1R based on attribute Siblings/Spouses Aboard: 0.651685393258427
Accuracy for 1R based on attribute Parents/Children Aboard: 0.5449438202247191
Accuracy for 1R based on attribute Fare: 0.6966292134831461



Depending on the Test / Train split we run here, this is either the second best attribute to pick is either Fare or Class. As the Fare is just dependent on the class (First class being more expensive than third, we will just pick the class as the second rule).
Once again, I write the rule function and evaluate it over the whole dataset.

In [9]:
def one_rule_class(row):
    if row['Pclass'] == 1:
        return 1
    return 0

predict_and_evaluate(one_rule_class)

Accuracy for one_rule_class: 0.677564825253664


0.677564825253664

# Task 2
Using your select stock / market index and your decision criterion (binary or ternary) on the daily return of the next day or on the trend (daily return after 5/10 days), can you generate a 1R model using as possible predictor the volume, and the moving average (with a period of 5, 10, 20 50 or 200). You can learn on all days except the last 100 (that will be used as the test set).

Let's clarify the task first:
- Selected market index: SMI
- decision criterion: Binary (Up or down)
- timeframe: daily.

In [10]:
smi = pandas.read_csv("Data/SMI.csv")
smi.describe()

,Open,High,Low,Close,Adj Close,Volume
count,7750.000000,7750.000000,7750.000000,7750.000000,7750.000000,7.750000e+03
mean,6467.866175,6506.270381,6427.279904,6467.621751,6467.621751,3.646313e+07
std,2495.146649,2505.374595,2483.327749,2494.479880,2494.479880,3.642250e+07
min,1288.699951,1296.599976,1279.000000,1287.599976,1287.599976,0.000000e+00
25%,5118.052368,5168.074829,5068.750122,5123.877563,5123.877563,0.000000e+00
50%,6683.669922,6721.635010,6644.150146,6681.550049,6681.550049,3.974700e+07
75%,8334.000244,8383.030273,8282.442871,8329.642334,8329.642334,5.766898e+07
max,12505.969727,12573.429688,12456.919922,12545.349609,12545.349609,3.467677e+08


In [11]:
smi.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1990-11-09,1378.900024,1389.000000,1375.300049,1387.099976,1387.099976,0
1,1990-11-12,1388.099976,1408.099976,1388.099976,1407.500000,1407.500000,0
2,1990-11-13,1412.199951,1429.400024,1411.400024,1415.199951,1415.199951,0
3,1990-11-14,1413.599976,1413.599976,1402.099976,1410.300049,1410.300049,0
4,1990-11-15,1410.599976,1416.699951,1405.099976,1405.699951,1405.699951,0
5,1990-11-16,1405.699951,1407.400024,1389.400024,1395.199951,1395.199951,0
6,1990-11-19,1395.599976,1417.900024,1395.599976,1416.000000,1416.000000,0
7,1990-11-20,1414.800049,1415.000000,1404.699951,1405.800049,1405.800049,0
8,1990-11-21,1405.599976,1405.599976,1396.699951,1398.400024,1398.400024,0
9,1990-11-22,1400.000000,1401.400024,1384.500000,1388.800049,1388.800049,0


From the Dataset Description we can see, that there seem to be a few missing values in the volume column. This must be kept in mind when moving forward.

Adding the daily return as a column in the dataset. We derive it using the closing price.
$r_{t} = \frac{p_{t} - p_{t-1}}{p_{t-1}}$


In [12]:
for index_smi in range(1,len(smi)):
    p_t = smi.loc[index_smi, 'Close']
    p_t_minus_1 = smi.loc[index_smi-1, 'Close']
    daily_return = (p_t - p_t_minus_1)/p_t_minus_1
    smi.loc[index_smi, 'Daily Return'] = daily_return

In [13]:
for index_smi in range(2,len(smi)):
    smi.loc[index_smi, 'Yesterday Daily Return'] = smi.loc[index_smi-1, 'Daily Return']

Building the ground truth labels

In [14]:
def build_ground_truth(row):
    if row['Daily Return'] > 0:
        return 1
    return 0

smi['Is Going Up'] = smi.apply (lambda row: build_ground_truth(row), axis=1)

Build the same model as above but don't do arbitrary test / validation splits.

In [15]:
x = smi[['Yesterday Daily Return']]
le = preprocessing.LabelEncoder()
y = smi['Is Going Up']
y = le.fit_transform(y)

x_train = x[2:-100]
y_train = y[2:-100]
x_test = x[-100:]
y_test = y[-100:]

one_r_2 = OneRClassifier()
one_r_2.fit(x_train.to_numpy(), y_train)
y_pred = one_r_2.predict(x_test.to_numpy())
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy for 1R based on yesterday daily return: {accuracy}')


Accuracy for 1R based on yesterday daily return: 0.4


Now we have a weird problem here. My OneRClassifier predicts all the time 0 (going down). Althoug it has been trained on data not only including going down.
I was unable to solve this problem. So here is a naive approach predicting a change to continue once it started. (Taking yesterdays trend and predicting it to continue)

In [16]:
def predict_same_as_yesterday(row):
    if row['Yesterday Daily Return']>0:
        return 1
    return 0

def is_correct_prediction_smi(row, prediction_name):
    if row['Is Going Up'] == row[prediction_name]:
        return True
    else:
        return False

smi['Prediction'] = smi.apply (lambda row: predict_same_as_yesterday(row), axis=1)
correct_predictions = smi.apply(lambda row: is_correct_prediction_smi(row, 'Prediction'), axis=1)
accuracy =  correct_predictions.value_counts()[True] / correct_predictions.size
print(f"Accuracy {accuracy}")

Accuracy 0.5054193548387097


And Thats a pretty bad accuracy. Almost the same as a coin toss...